In [ ]:
!nvidia-smi

Wed Dec 27 01:21:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 7.7 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"


In [ ]:
import os
import random
import shutil
from tqdm import tqdm

def train_test_split(path, neg_path=None, split=0.2, train_path_img='/content/train/images', train_path_label='/content/train/labels', val_path_img='/content/val/images', val_path_label='/content/val/labels'):
    print("------ PROCESS STARTED -------")

    files = list(set([name[:-4] for name in os.listdir(path)]))

    print(f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        shutil.copy2(os.path.join(path, filex + '.jpg'), os.path.join(train_path_img, filex + '.jpg'))
        shutil.copy2(os.path.join(path, filex + '.txt'), os.path.join(train_path_label, filex + '.txt'))

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)]))
        for filex in tqdm(neg_images):
            shutil.copy2(os.path.join(neg_path, filex + ".jpg"), os.path.join(train_path_img, filex + '.jpg'))

        print(f"------ Total {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")

    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        shutil.copy2(os.path.join(path, filex + '.jpg'), os.path.join(val_path_img, filex + '.jpg'))
        shutil.copy2(os.path.join(path, filex + '.txt'), os.path.join(val_path_label, filex + '.txt'))

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    print("------ TASK COMPLETED -------")

# Modify the paths as needed
train_test_split('/content/drive/MyDrive/Malaayeeka_YOLOv8Final/Data')  # without negative images
# train_test_split('/content/drive/MyDrive/Malaayeeka/Data', '/content/drive/MyDrive/negative_images')  # if you want to feed negative images


------ PROCESS STARTED -------
--- This folder has a total number of 1702 images---


100%|██████████| 1362/1362 [00:48<00:00, 28.20it/s] 


------ Training data created with 80% split 1362 images -------


100%|██████████| 340/340 [00:01<00:00, 241.72it/s]

------ Testing data created with a total of 340 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/78.2 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Malaayeeka_YOLOv8Final/data.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/Malaayeeka_YOLOv8Final/Training_results name=WeaponDetection save=true

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Malaayeeka_YOLOv8Final/data.yaml, epochs=10, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Malaayeeka_YOLOv8Final/Training_results, name=WeaponDetection2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Malaayeeka_YOLOv8Final/Training_results/WeaponDetection2/weights/best.pt conf=0.35 source=/content/val/images save=true

Ultralytics YOLOv8.0.230 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs

image 1/340 /content/val/images/00917d05-290_jpeg_jpg.rf.61264ab2efc5d1f1b211ded80a032028.jpg: 640x640 (no detections), 16.3ms
image 2/340 /content/val/images/0608879c-93_jpeg_jpg.rf.2e80811d3a5f3617c87d9154a857cd16.jpg: 640x640 (no detections), 22.2ms
WARNING ⚠️ NMS time limit 0.550s exceeded
image 3/340 /content/val/images/069ad4de-ep114_jpg.rf.b73e38fc3dae8852fa287242f1362608.jpg: 640x640 1 0, 16.4ms
image 4/340 /content/val/images/06f9b400-e1339_jpg.rf.5663cb5fc6291269111a4bf14340a766.jpg: 640x640 2 0s, 16.5ms
image 5/340 /content/val/images/07ab0e6c-images_12_jpeg_jpg.rf.87dfd7fe2fe3961963731a265c2136e4.jpg: 640x640 1 0, 16.3ms
image 6/340 /content/val/images/08225443-ep252_jpg.rf.50d0ce8c4093d28cb9280d727595a11c.jpg: 640x640 1 0, 16.3ms
image 7/340 /content/val/images/09eede30-219_jpeg_jpg.rf.2230e939b0a41ba9a542

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/Malaayeeka_YOLOv8Final/Output

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Malaayeeka_YOLOv8Final/Training_results/WeaponDetection2/weights/best.pt conf=0.35 source=/content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4 save=true

Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (1/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 5 players, 98.2ms
video 1/1 (2/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 5 players, 11.1ms
video 1/1 (3/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 6 players, 1 football, 11.0ms
video 1/1 (4/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 5 players, 1 football, 11.0ms
video 1/1 (5/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 8 players, 11.0ms
video 1/1 (6/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 8 players, 11.0ms
video 1/1 (7/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 8 players, 11.0ms
video 1/1 (8/978) /content/drive/MyDrive/Malaayeeka/Videos/Highlights.mp4: 384x640 8 players, 11.0m

In [ ]:
!cp -r /content/runs/detect/predict3 /content/drive/MyDrive/Malaayeeka/Output
